<a href="https://colab.research.google.com/github/unclepeddy/tf-serving-fun/blob/master/examples/keras-lstm-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
# Specify hyperparams

max_features = 10000
maxlen = 500
batch_size = 32
lstm_width = 32
embedding_dim = 32
epochs = 1 

In [0]:
# Build and compile a simple LSTM model

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_features)

train_data = tf.keras.preprocessing.sequence.pad_sequences(
  train_data, maxlen=maxlen)
test_data = tf.keras.preprocessing.sequence.pad_sequences(
  test_data, maxlen=maxlen)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_features, embedding_dim))
model.add(tf.keras.layers.LSTM(lstm_width))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [11]:
# Train model using a slice of training data
train_data = train_data[:100]
train_labels = train_labels[:100]

history = model.fit(train_data, train_labels, 
	epochs = 1,
	batch_size=batch_size,
	validation_split=0.2)

Train on 80 samples, validate on 20 samples
80/80 [==============================] - 2s 22ms/sample - loss: 0.6959 - acc: 0.3875 - val_loss: 0.6934 - val_acc: 0.5500


In [0]:
# Export model

import os 
model_name = 'lstm_model'
model_base_path = os.path.join(os.getcwd(), model_name)
model_version = 0
model_version_path = os.path.join(model_base_path, str(model_version))

os.environ["MODEL_NAME"] = model_name
os.environ["MODEL_DIR"] = os.path.abspath(model_base_path)
os.environ["MODEL_VERSION_DIR"] = os.path.abspath(model_version_path)

!rm -rf $MODEL_DIR

# Enable to strip default op attributes
enable_forward_compat = True

if (enable_forward_compat):

  prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={'input': tf.saved_model.build_tensor_info(model.input)},
    outputs={t.name: tf.saved_model.build_tensor_info(t) for t in model.outputs},
    method_name= tf.saved_model.signature_constants.PREDICT_METHOD_NAME
  )

  builder = tf.saved_model.Builder(model_version_path)
  builder.add_meta_graph_and_variables(
    tf.keras.backend.get_session(),
    [tf.saved_model.tag_constants.SERVING],
    signature_def_map = {
      tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature
    },
    strip_default_attrs=True
  )

  builder.save()

else:
  tf.saved_model.simple_save(
    tf.keras.backend.get_session(),
    model_version_path, 
    inputs={'input': model.input},
    outputs={t.name:t for t in model.outputs}
  )


In [0]:
# Explore the model signature
!saved_model_cli show --dir $MODEL_VERSION_DIR --all

In [0]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server


In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name="$MODEL_NAME" \
  --model_base_path="$MODEL_DIR" >server.log 2>&1

In [0]:
! cat server.log

In [98]:
# Query your model's status
!curl http://localhost:8501/v1/models/$MODEL_NAME

{
 "model_version_status": [
  {
   "version": "0",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


In [208]:
# Query the model
! pip install -q requests
import json, requests

samples = test_data[0:2].tolist()
request_data = json.dumps({"signature_name": "serving_default", "instances": samples})
headers = {"content-type" : "application/json"}
server_url = "http://localhost:8501/v1/models/{}:predict".format(model_name) 

json_response = requests.post(server_url, data=request_data, headers=headers)
pred = json.loads(json_response.text)['predictions']

print(pred)

[[0.498789], [0.500337]]
